In [2]:
import math
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [3]:
## Thermodynamic Ks
# Below is a list of all the thermodynamic equilibrium constants used in the model as 
# functions of T where possible. We later use activity coefficients
# determined using our pitzer and empirical values  model to convert to
# conditional constants 
 
## SOURCES

# MILLERO & PIERROT 1998 
##  Table I. Coefficients for the thermodynamic constants for the dissociation of acids [0 to 50 C for all of the acids and to higher temperatures for some]
T_param_HF      = array([-12.641, 1590.2, 0, 0])
T_param_HSO4    = array([141.411, -4340.704, -23.4825, 0.016637])

## Table II. Coefficients for the formation of ion pairs
T_param_MgOH    = array([3.87, -501.6, 0])
T_param1_MgF    = array([3.504, -501.6, 0])
T_param1_CaF    = array([3.014, -501.6, 0])
T_param1_MgCO3  = array([1.028, 0, 0.0066154])      # k25MgCO3  = 2.98;   delta_h = 2.713 kcal;  analytical = [0.9910, 0.00667, 0.0, 0.0, 0.0 ,0.0]
T_param1_CaCO3  = array([1.178, 0, 0.0066154])      # k25CaCO3  = 3.224;  delta_h = 3.545 kcal;  analytical = [-1228.732, -0.299444, 35512.75, 485.818, 0.0, 0.0]
T_param_SrCO3   = array([1.028, 0, 0.0066154])
# param_MgH2PO4 = [1.13, 0, 0];
# param_CaH2PO4 = [-1, 0, 0];
# param_MgHPO4  = [2.7, 0, 0];
# param_CaHPO4  = [2.74, 0, 0];
# param_MgPO4   = [5.63, 0, 0];
# param_CaPO4   = [7.1, 0, 0];

## SHIAMI DATABASE FROM LLNL/THERMO.COM.V8.R6+.tdat
# Datasets: https://www.gwb.com/thermo.php

## ------------------------------------------------------------------------------------------------

# 1) LLNL/THERMO.COM.V8.R6+.tdat
# 
# 'The "THERMO.COM.V8.R6+.tdat"  file is the expanded version of the LLNL database which 
# includes log Ks for  hundreds of reactions involving aqueous species, minerals, gases,  many organic species and 
# radionuclides. It also contains coefficients for evaluating activity coefficients by the B-dot equation 
# but this is not used in SHIAMI. Some people feel this database is less internally consistent than thermo.tdat, 
# especially with respect to aluminum and sulfur species.' [https://www.gwb.com/thermo.php]

# The 'llnl. dat' file is a database provided with the PHREEQC program. It is essentially 
# the same as 'thermo.com.V8.R6.230' however it has been converted to Phreeqc format by 
# Greg Anderson with help from David Parkhurst.  Delta H of reaction are calculated from Delta H of 
# formations given in thermo.com.V8.R6.230 (8 Mar 2000).


# 2) phreeqc.dat/wateq4f.dat

# These datasets are thermodynamic databases derived from Wateq4F.
# The "Phreeqc.dat" is a database file that is consistent with wateq4f.dat, but 
# has a smaller set of elements and aqueous species.  The dataset is provided with PhreeqC [USGS]  
# Obtained from GWB: "https://www.gwb.com/thermo.php"

## ------------------------------------------------------------------------------------------------

# The log Ks provided are thermodynamic association constants for infinite dilution. 
# The log K is assumed to apply at all ionic strengths because they are in terms of activities. 
# All ionic strength corrections go into the activity coefficients of the aqueous species.
# 
# Effectively, PHREEQC automatically uses: 
#         log K(conditional) = log K(thermodynamic) * gamma(reactants)/gamma(products) 

# If you use conditional constants for seawater, the mass action equations  are written with 
# molalities rather than activities and all ionic strength corrections go into the log K. 

# The enthalpy of reaction is used in the Vant Hoff equation to determine the temperature 
# dependence of the equilibrium constant. Internally, all enthalpy calculations are performed 
# in the units of kJ/mol. Default units are kJ/mol. 

# For some species, analytical expression for the temperature dependence of log K is defined. 
# If defined, the analytical expression takes precedence over log_k and the Vant Hoff equation 
# to determine the temperature dependence of the equilibrium constant.  



## Na
T_param_NaCl    = array([1.1398e2, 3.6386e-2, -3.0847e3, -4.6571e1, -4.8167e1, 0.0])     # [-0.777, 5.21326]   kJ/mol;   Range:  0-300
T_logk25NaSO4   = array([0.70, 1.12*4.184*1000])                                         # delta_h = 1.12      kcal;     delta h from phreeqc   log from llnl.dat                    
T_logk25NaCO3   = array([1.27, 8.91*4.184*1000])                                         # delta_h = 8.91      kcal;     phreeqc                         
T_logk25NaHCO3  = array([-0.25, -13.7741*1000])                                          # [0.1541, -13.7741]  kJ/mol;   Range:  0-200             %  [-9.0668e1,-2.9866e-2, 2.7947e3, 3.6515e1, 4.7489e1, 0.0]              
T_param_NaBr    = array([1.1871e2, 3.7271e-2, -3.4061e3, -4.8386e1, -5.3184e1,0.0])      # [-1.3568, 6.87431] kJ/mol;   Range:  0-300 
T_param_NaF     = array([1.2507e2, 3.8619e-2, -3.5436e3, -5.0787e1, -5.5332e1, 0.0])     # [-0.9976, 7.20903]  kJ/mol;   Range:  0-300       
T_k25NaB        = 10**0.22                                                               # MILLERO & SCHREIBER 1982
T_k25NaOH       = 10**-0.18                                                              # MILLERO & SCHREIBER 1982 

## K
T_param_KCl     = array([1.3650e2, 3.8405e-2, -4.4014e3, -5.4421e1, -6.8721e+1, 0.0])    # [-1.4946, 14.1963]  kJ/mol;  Range:  0-300                         
T_param_KSO4    = array([3.106, 0.0, -673.6, 0.0, 0.0, 0.0])                             # [9.9073e1, 3.7817e-2, -2.1628e3, -4.1297e1, -3.3779e1, 0.0];     % [0.8796, 2.88696]   kJ/mol;  Range:  0-300                      
T_param_KBr     = array([1.1320e2, 3.4227e-2, -3.6401e3, -4.5633e1, -5.6833e1, 0.0])     # [-1.7372, 12.5102]  kJ/mol;  Range:  0-300                                                        
# k25KOH        = 10^-14.46;                                                             # MILLERO & SCHREIBER 1982 

## Mg
T_param_MgCl    = array([4.3363e1, 3.2858e-2, 1.1878e+2, -2.1688e+1, 1.8403e+000, 0.0])  # [-0.1349, -0.58576] kJ/mol;  Range:  0-300                      
T_param_MgSO4   = array([1.7994e2, 6.4715e-2, -4.7314e3, -7.3123e1, -8.0408e1, 0.0])     # [2.4117, 19.6051]   kJ/mol;  Range:  0-200                                            
T_param2_MgCO3  = array([0.9910, 0.00667, 0.0, 0.0, 0.0 ,0.0])                           # [2.98, 2.713]         kcal;  phreeqc
T_param_MgHCO3  = array([3.8459e1, 3.0076e-2, 9.8068e1, -1.8869e1, 1.5187e+000, 0.0])    # [1.0357, 2.15476]   kJ/mol;  Range:  0-300                                                      
T_k25MgF        = array([1.82, 3.2*4.184*1000])                                          # [1.82, 3.2]         kcal;    [6.4311e1, 3.5184e-2, -7.3241e2, -2.8678e1, -1.1448e1, 0.0];     % [1.3524, 2.37233]   kJ/mol;  Range:  0-300                      
T_k25MgB        = 10**1.57                                                               # MILLERO & SCHREIBER
# param_MgOH = SEE ABOVE                                                                 # MILLERO & PIERROT 1998

## Ca 
T_param_CaCl    = array([8.1498e1, 3.8387e-2, -1.3763e3, -3.5968e1, -2.1501e1, 0.0])     # [-0.6956, 2.02087]  kJ/mol;   Range:  0-300                        
T_k25CaSO4      = array([2.3, 1.65*4.184*1000]) # kcal                                   # [2.8618e2, 8.4084e-2, -7.6880e3, -1.1449e2, -1.2005e2, 0.0];    % [2.1111, 5.4392]    kJ/mol;   Range:  0-300                     
T_param2_CaCO3  = array([-1228.732, -0.299444, 35512.75, 485.818, 0.0, 0.0])             # [3.224, 3.545]      kcal;     PHREEQC
T_param_CaHCO3  = array([5.5985e1, 3.4639e-2, -3.6972e2, -2.5864e1, -5.7859, 0.0])       # [1.0467, 1.45603]   kJ/mol;   Range:  0-300                                             
T_k25CaF        = array([0.94, 4.12*4.184*1000])                                         # [7.8058e1, 3.8276e-2, -1.3289e3, -3.4071e1, -2.0759e1, 0.0];    % [0.6817, 5.6484]    kJ/mol;   Range:  0-300                         
T_k25CaB        = 10**1.80                                                               # MILLERO & SCHREIBER
T_k25CaOH       = 10**1.3                                                                # MILLERO & SCHREIBER  

## Sr
T_param_SrCl    = array([9.4568e1, 3.9042e-2, -2.1458e3, -4.0105e1, -3.3511e1, 0.0])     # [-0.2485, 7.58559]  kJ/mol;   Range:  0-300                         
T_logk25SrSO4   = array([2.29, 2.08*4.184*1000])                                         # delta_h = 2.08      kcal;     phreeqc         
# param_SrCO3 =  SEE ABOVE                                                               # MILLERO
T_param_SrHCO3  = array([-3.248, 0.014867, 0.0, 0.0, 0.0, 0.0])                          # [1.18, 6.05]        kcal;     phreeqc                         
T_param_SrF     = array([9.0295e1, 3.7609e-2, -1.9012e3, -3.8379e1, -2.9693e1, 0.0])     # [0.1393, 4.8116]    kJ/mol;   Range:  0-300                        
T_k25SrB        = 10**1.55                                                               # MILLERO & SCHREIBER
T_k25SrOH       = 10**0.82                                                               # MILLERO & SCHREIBER

## MINOR CATION

## H

## Li
T_param_LiCl    = array([1.2484e2, 4.1941e-2, -3.2439e3, -5.1708e1, -5.0655e1, 0.0])     # [-1.5115, 3.36812]  kJ/mol   Range:  0-300	
T_k25LiSO4      = 10**0.7
T_param_LiF     = array([0.310, -4.00*1000]) # kJ?/mol
# k25LiOH =10^0.3599

## Rb
T_param_RbCl    = array([1.2689e2, 3.5557e-2, -4.0822e3, -5.0412e1, -6.3736e1, 0.0])     # [-0.9595, 13.1922]	kJ/mol  Range:  0-300
T_k25RbSO4      = 10**0.94
T_param_RbBr    = array([1.2054e2, 3.3825e-2, -3.9500e3, -4.7920e1, -6.1671e1, 0.0])     # [-1.2168, 13.9327]	kJ/mol  Range:  0-300
T_param_RbF     = array([1.3893e2, 3.8188e-2, -3.8677e3, -5.5109e1, -6.0393e1, 0.0])     # [+0.9602 1.92464]	kJ/mol; Range:  0-300

## Cs
T_k25CsBr       = array([1.2064e2, 3.2000e-2, -3.8770e3, -4.7458e1, -6.0533e1, 0.0])     # [-0.2712, 10.9621]	kJ/mol	Range:  0-300
T_k25CsCl       = array([1.2472e2, 3.3730e-2, -3.9130e3, -4.9212e1, -6.1096e1,0.0])      # [-0.1385, 2.73215]	kJ/mol	Range:  0-300
T_k25CsF        = 10**-0.3600 
T_k25CsSO4      = 10**1.0400
T_k25CsCO3      = 10**-0.2900

## NH4
T_k25NH4SO4     = 10**1.11   # WATEQ4f

## Ba
T_param_BaCl    = array([1.1016e+2, 4.2325e-2, -2.8039e+3, -4.6010e+1, -4.3785e+1, 0.0])  # [-0.4977, 11.142]	kJ/mol	Range:  0-300
T_param_BaF     = array([1.0349e2, 4.0336e-2, -2.5195e3, -4.3334e1, -3.9346e1,0.0])       # [-0.1833, 8.95376] kJ/mol	Range:  0-300
T_k25BaSO4      = 10**2.7 
T_param_BaHCO3  = array([-3.0938, 0.013669, 0.0, 0.0, 0.0, 0.0])                          # [0.982, 5.56] kcal  # WATEQ4f
T_param_BaCO3   = array([0.113, 0.008721, 0.0, 0.0, 0.0, 0.0])                            # [2.71, 3.55] kcal   # WATEQ4f

## Minor Anions

## HSO4

## HS

## I
T_param_NaI     = array([9.8742e1, 3.2917e-2, -2.7576e+3, -4.0748e1, -4.3058e1, 0.0])     # [1.54, 7.33455] kJ/mol	Range:  0-300
T_param_KI      = array([1.0816e2, 3.3683e-2, -3.2143e+3, -4.4054e1, -5.0187e1, 0.0])     # [ -1.598, 9.16296] 	kJ/mol	Range:  0-300

## ClO3

## ClO4

## BrO3

## CNS

## NO2

## NO3

## H2PO4